# 概要
tensorflow是一个强有力的数值计算库，特别适应大规模的机器学习任务。  
它的规则很简单，首先定义一组python的计算图，然后TensorFlow使用优化的C++代码高效的计算和运行。  
更重要的是，可以将图计算分为若干块，并行的在CPUs和GPUs内计算，TensorFlow支持分布计算，可以将超大规模的训练集分为上千的服务器去计算并训练大规模的神经网络，可以使用百万级的样例，百万级的特征，训练百万级参数的神经网络。还有很多优点:   
1.可以运行在多种平台，甚至移动平台。  
2.提供简单的API叫做TF.contrib.learn，兼容sklearn。  
3.提供TF-silm来简化建立、训练和评估神经网络；  
4.一些高级的API是基于TF的比如Keras或者Pretty Tensor。  
...  
# 创建第一个图并运行

In [1]:
import tensorflow as tf

x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')
f = x*x*y + y + 2

sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)

42


session是将图放在运算器上运行并且保存数据，可以使用如下方式运行：

In [2]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
print(result)

42


在上面的with块中，tf.Session()是默认的session，调用x.initializer.run()等价于tf.get_default_session().run(x.initializer)。  
  
还有一种运行方式。可以使用global_variable_initialier()。会在计算运行的时候自动载入数值：

In [3]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    result = f.eval()
print(result)

42


在jupyter中还可以使用InteractiveSession。和一般的session不同，它会自动的创建session并作为默认值，所以都不需要block，但是注意最后要关闭他

In [4]:
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)
sess.close()

42


至此，我们可以看到tf编程有两个组成部分，一个是建立计算图，称为构造层construction phase；一个是运行它，称为执行层execution phase。  
通常对于ML任务来说，构造层用来表达ML模型和一些需要的计算，执行层用来运行一些循环来重复训练步骤，或者更新参数。

# 管理图
任何新建的节点都会自动加入默认的图:

In [5]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

大多数情况下是不错的，但是有时可能需要管理多个单独的图，可以新建图然后暂时的将它作为默认图加入到with中

In [6]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)
print(x2.graph is graph)
print(x2.graph is tf.get_default_graph())

True
False


在Jupyter中，可能同样的语句会重复执行很多遍，导致默认图里有很多重复节点，一个办法是重启，重新执行，另一个简便的办法是重载默认图tf.reset_default_graph()

# 节点变量的生命周期
当求一个节点的值时，tf会自动的确定一系列与所求节点值需要的节点，并先计算他们。

In [7]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

10
15


在上面的例子里，代码首先定义了一个简单的图，然后他开始了一个session并且计算y。tf先检测到y是需要x和w的值的，所以先计算他们的值，最后在返回计算y的值。最后计算z的值，又重新再检测到需要x和w的值，所以再次计算两者的值。所以上述过程中x和w的值被计算两次。  
在图运行过程中所有节点的除了变量的值都会被忽略，而变量的值保存在session.变量的生命周期就是他初始化直到session关闭。   
如果想要高效的计算y和z,可以在一次图计算中计算两者的值：

In [8]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y.eval())
    print(z.eval())

10
15


在单进程的tf中，多个sess之间是无法共享内容的，即使使用同一个图(每个session有自己的变量副本)。在分布式tf中，变量保存在自己的服务器里，而不是在session中，所以可以共享。
# 使用tf实现线性回归
tf可以处理多输入和任意数量的输出，输入和输出都是多维数组，称为tensors。和numpy的array一样，有type和shape。  

In [9]:
import numpy as np
from sklearn.datasets import fetch_california_housing


housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')

XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()
    
theta_value

array([[-3.72048187e+01],
       [ 4.36271667e-01],
       [ 9.38740931e-03],
       [-1.07066855e-01],
       [ 6.44966364e-01],
       [-4.11580004e-06],
       [-3.77993658e-03],
       [-4.23913479e-01],
       [-4.37445879e-01]], dtype=float32)

最主要的好处就是可以在GPU上运行。
# 使用梯度下降
下面我们将使用梯度下降算法实现，首先先展示手动计算梯度，然后使用tf分autodiff特征让tf自动计算梯度，最后我们使用tf的out-of-box优化器
## 手动计算梯度
使用random_uniform()函数创建一个能产生随机数的节点，给出数组型和数值范围，类似numpy的rand()；  
使用assign()函数创建一个节点，用来分配新的变量，如batch的每步theta    

In [10]:
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_housing_data_plus_bias = scaler.fit_transform(housing_data_plus_bias)

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='error')
gradients = 2/m*tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print('Epoch', epoch, 'MSE =', mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()
    
print(best_theta)

Epoch 0 MSE = 8.972075
Epoch 100 MSE = 5.0195365
Epoch 200 MSE = 4.907174
Epoch 300 MSE = 4.876695
Epoch 400 MSE = 4.8577304
Epoch 500 MSE = 4.843993
Epoch 600 MSE = 4.8338566
Epoch 700 MSE = 4.826343
Epoch 800 MSE = 4.820754
Epoch 900 MSE = 4.8165817
[[ 0.43165946]
 [ 0.859406  ]
 [ 0.15429659]
 [-0.26576626]
 [ 0.28124905]
 [ 0.00804014]
 [-0.04296798]
 [-0.612791  ]
 [-0.5842021 ]]


## 使用autodiff
前面的代码运行良好，但是要从MSE上计算得到梯度，在线性回归的情况下，这个是容易求解的，但是在神经网络中很难进行，主要是不方便求解，且容易出错。在没有要求非常效率的情况下。可以使用symbolic differentiation符号微分来自动得到偏导等式。  
考虑$f(x)=e^{e^{e^x}}$,他的导数是$f'(x)=e^xe^{e^x}e^{e^{e^x}}$,如果代码中将他们分别都计算出来，显然是不够效率的。  
一个效率的办法是计算$e^x$,再计算$e^{e^x}$,最后计算$e^{e^{e^x}}$,这样既得到了$f(x)$，也通过三个乘积得到$f'(x)$。  
根据这个基本想法，可以将9次指数计算减少为3次。  
但是函数为任意情况下，就不容易寻找这样的偏导数，比如这样:

In [11]:
def my_fun(a, b):
    z = 0
    for i in range(100):
        z = a * np.cos(z + i) + z * np.sin(b - i)
    return z

好消息是，tf的autodiff特征可以解决这个问题，自动高效地求出梯度，简单的替换掉前面的梯度函数，仅仅使用下面的代码：

In [12]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_housing_data_plus_bias = scaler.fit_transform(housing_data_plus_bias)

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name='theta')

y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='error')

#gradients = 2/m*tf.matmul(tf.transpose(X), error)

gradients = tf.gradients(mse, [theta])[0]

training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print('Epoch', epoch, 'MSE =', mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()
    
print(best_theta)

Epoch 0 MSE = 7.3364763
Epoch 100 MSE = 5.2583647
Epoch 200 MSE = 5.0591235
Epoch 300 MSE = 4.9846625
Epoch 400 MSE = 4.935235
Epoch 500 MSE = 4.8996534
Epoch 600 MSE = 4.873806
Epoch 700 MSE = 4.854994
Epoch 800 MSE = 4.841282
Epoch 900 MSE = 4.831274
[[-0.366812  ]
 [ 0.82088834]
 [ 0.16848813]
 [-0.15294155]
 [ 0.17033917]
 [ 0.01370264]
 [-0.04334717]
 [-0.5455075 ]
 [-0.510498  ]]


这里gradients()函数操作一个op(这里是mse)和一个变量列表(这里是theta)，并且建立一个列表的ops来计算op对每个列表变量的偏导数，所以gradients节点将计算一个偏导向量,tf使用reverse-mode autodiff，效率和准备度在多输入少输出情况下很理想，常用于神经网络，它计算偏导向量需要遍历$n_{output}+1$次图。

## 使用优化器
所以tf可以直接计算梯度，但是这还早，tf还提供了一系列的优化器，包括了梯度下降优化器。  

In [13]:
#gradients = 2/m*tf.matmul(tf.transpose(X), error)
gradients = tf.gradients(mse, [theta])[0]

#training_op = tf.assign(theta, theta - learning_rate * gradients)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print('Epoch', epoch, 'MSE =', mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()
    
print(best_theta)

Epoch 0 MSE = 8.908798
Epoch 100 MSE = 4.9779143
Epoch 200 MSE = 4.8793097
Epoch 300 MSE = 4.8600364
Epoch 400 MSE = 4.8470325
Epoch 500 MSE = 4.837239
Epoch 600 MSE = 4.829777
Epoch 700 MSE = 4.82406
Epoch 800 MSE = 4.8196564
Epoch 900 MSE = 4.8162475
[[ 0.6451824 ]
 [ 0.8928189 ]
 [ 0.15245952]
 [-0.34404847]
 [ 0.3526465 ]
 [ 0.00693861]
 [-0.04352513]
 [-0.5948129 ]
 [-0.57084596]]


如果想要使用别的优化器只需要更改一行就行了。

# 给算法提供数据
我们尝试把前面的算法修改为Mini-batch形式。我们需要把X,y替换为mini-batch的迭代器形式，最简方法就是修改节点，使用placeholder节点，这类节点的特点是不实际执行任何计算，它们只在实际运行的时候输出用户想要输出的值，典型应用在tf训练模型的时候提供训练数据。如果不将节点变为placeholder节点，运行时将会弹出exception。  
  
建立placeholder节点，需要调用placeholder()，并且设置输出tensor的数据类型，也可以设置数组形状，这项的默认值是any size。比如，下面的代码建立了placeholder节点A，以及一个节点B=A+5.当我们求B得值时，我们将对eval()使用feed_dict来特例化A的值，注意A有2级，即2维,但是可以是任意级，以及3列，必须是3列：

In [14]:
A = tf.placeholder(tf.float32, shape=(None, 3))
B = A + 5
with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A: [[1, 2, 3]]})
    B_val_2 = B.eval(feed_dict={A: [[4, 5, 6], [7, 8, 9]]})

print(B_val_1)
print(B_val_2)

[[6. 7. 8.]]
[[ 9. 10. 11.]
 [12. 13. 14.]]


为了实施mini-batch，应该如下:

In [15]:
X = tf.placeholder(tf.float32, shape=(None, n + 1), name='X')  
y = tf.placeholder(tf.float32, shape=(None, 1), name='y')  
   
batch_size = 100 
n_batch = int(np.ceil(m / batch_size))  
  
def fetch_batch(epoch, batch_index, batch_size):  
    X_batch = scaled_housing_data_plus_bias[epoch * batch_index : epoch * batch_index + batch_size]
    y_batch = housing.target.reshape(-1, 1)[epoch * batch_index : epoch * batch_index + batch_size]
    return X_batch, y_batch  
  
with tf.Session() as sess:  
    sess.run(init)
    
    for batch_index in range(batch_size):  
        X_batch, y_batch = fetch_batch(epoch, batch_index , batch_size)  
        sess.run(training_op, feed_dict={X: X_batch, y: y_batch})  
    best_theta = theta.eval()  
      

# 保存及重载模型
一旦训练好模型，便可以将他们保存在硬盘中，用来应用到其他程序或是比较模型。或者是保存进度以免计算机故障而前功尽弃。  
tf保存和重载模型十分简单，只需要在构造层的末尾创建一个 saver节点，然后执行层调用save()函数：  

In [16]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            save_path = saver.save(sess, "/tmp/my_model.ckpt")
        
        sess.run(training_op)
    
    best_theta = theta.eval()
    save_path = saver.save(sess, "/tmp/my_model_final.ckpt")

重载模型只需要在构造层的最后加入saver，然后在执行层的最前面用restore()替换init

In [17]:
with tf.Session() as sess:
    saver.restore(sess, '/tmp/my_model_final.ckpt')
    best_theta = theta.eval()

INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt


默认情况下saver使用自己的名字保存和重载所有变量，也可以自己更改：  
saver = tf.train.Saver({'weights': theta})

# TensorBoard图的可视化和训练曲线
第一步是微调程序，写入图的定义和一些状态量到日志中，这样tfBoard可以读取它们。需要使用不同的日志名当编程的时候。最佳办法就是加入时间戳

In [18]:
from datetime import datetime

now = datetime.utcnow().strftime('%Y%m%d%H%M%S')
root_logdir = 'tf_Logs'
logdir = "{}/run-{}/".format(root_logdir, now)

#然后将下列函数加入构造层末尾  
mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

第一行在图中创建了一个求MSE并将其写入名为summary的TsBoard-兼容的二进制日志的节点。第二行创建了一个FileWriter，用来将summary写入到指定日志文件中，第一个参数是日志路径，第二个是用来可视化的图。这里的结果就是创建一个原来不存在的日志路径并且将图写入到一个名为events file事件文件的二进制文件中。  
下面就是更新执行层，并且对mse_summary节点在训练时求值

In [19]:
with tf.Session() as sess:
    sess.run(init)
    
    for batch_index in range(n_batch):
        X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
        if batch_index % 10 == 0: #避免每步都保存降低训练效率
            summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
            step = epoch * n_batch +batch_index
            file_writer.add_summary(summary_str, step)
        sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

#最后关闭FileWriter
file_writer.close()

上面代码创建了一个日志路径，并且把事件写入，包括图定义和MSE的值。  
现在可以打开TSboard服务器了。需要在环境中激活它，然后使用tensorboard命令打开它，下面是开启sever，监听在6006端口  
\$ source env/bin/activate  
\$ tensorboard --logdir tf_logs/  
Starting TensorBoard on port 6006  

# 名字域

在处理复杂模型，比如神经网络时，图可能会聚合成一堆，其中有上万个节点，为了避免这种情况，可以使用name scopes来把相关的图放在一起，比如我们把前面的error 和mse放在一个叫做loss的名字域中，

In [20]:
with tf.name_scope('loss') as scope:
    error = y_pred - y 
    mse = tf.reduce_mean(tf.square(error), name='mse')

# 模块化
假如想要建立一个图，其中输出是两个修正线性单元rectified lineear units(ReLU).一个ReLU计算输入的线性函数，如果结果是正就输出正数，否则输出0：  
$h_{\mathbf w,b}(\mathbf X) = max(\mathbf X\cdot\mathbf w+b,0)$  
下面是这个单元的代码：  

In [21]:
n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name='X')

w1 = tf.Variable(tf.random_normal((n_features, 1)), name='weights1')
w2 = tf.Variable(tf.random_normal((n_features, 1)), name='weights2')
b1 = tf.Variable(0.0, name='bias1')
b2 = tf.Variable(0.0, name='bias2')

z1 = tf.add(tf.matmul(X, w1), b1, name='z1')
z2 = tf.add(tf.matmul(X, w2), b2, name='z2')

relu1 = tf.maximum(z1, 0., name='relu1')
relu2 = tf.maximum(z2, 0., name='relu2')

output = tf.add(relu1, relu2, name='output')

这样的代码含有大量重复。不易维护而且易出错。特别是想增加更多这类单元的时候，tf可以让你原理DRY(don't repeat yourself自我重复)：简单的构造一个模块:

In [22]:
def relu(X):
    w_shape = (int(X.get_shape()[1]), 1)
    w = tf.Variable(tf.random_normal(w_shape), name='weights')
    b = tf.Variable(0.0, name='bias')
    z = tf.add(tf.matmul(X, w), b, name='z')
    return tf.maximum(z, 0., name='relu')

n_features = 3
X = tf.placeholder(tf.float32, shape=((None, n_features)), name='X')
relu = [relu(X) for i in range(5)]
output = tf.add_n(relu, name='output')

tf自动检测重名，并且加入后缀,这样tf定义一系列的变量来减少簇。

In [23]:
#然后将下列函数加入构造层末尾  
sess = tf.InteractiveSession()
file_writer = tf.summary.FileWriter(logdir, sess.graph)
init.run()

sess.close()
#最后关闭FileWriter
file_writer.close()

# 共享变量
如果想要在一个图之间的若干部位共享变量，一个简单的方法是创建他，然后将它作为函数的参数，比如你想要控制ReLU的阈值，在所有的ReLU间共享阈值，可以如下:

In [24]:
def relu(X, threshold):
    with tf.name_scope('relu'):
        w_shape = (int(X.get_shape()[1]), 1)
        w = tf.Variable(tf.random_normal(w_shape), name='weights')
        b = tf.Variable(0.0, name='bias')
        z = tf.add(tf.matmul(X, w), b, name='z')
        return tf.maximum(z, threshold, name='max')
    
tf.reset_default_graph()
sess = tf.InteractiveSession()
threshold = tf.Variable(0.0, name='threshold')
X = tf.placeholder(tf.float32, shape=(None, n_features), name='X')
relus = [relu(X,threshold) for i in range(5)]
output = tf.add_n(relus, name='output')

file_writer = tf.summary.FileWriter(logdir, sess.graph)

sess.close()
file_writer.close()

这个结果看着不错，现在可以对所有ReLU的阈值进行调整，然而，如果有很多歌这样的共享变量，操作起来也很麻烦。tf提供一个选项，可以比较清晰地解决。思路是使用get_variable()函数来创建一个共享变量，不管他是否真实存在，想要的操作行为由当前的vaiable_scope()的特性操控。比如说下面的代码创建了一个relu/threshold变量：

In [25]:
with tf.variable_scope('relu'):
    threshold = tf.get_variable('threshold', shape=(),
                               initializer=tf.constant_initializer(0.0))

注意这里如果变量在get_variable前已经创建，代码将会弹出异常。这种方式防止错误的重用变量，如果想要重用，需要在scope中设置reuse属性为真(这种情况下不需要特例化数组型和初始化):

In [26]:
with tf.variable_scope('relu', reuse=True):
    threshold = tf.get_variable('threshold')

这个代码将会取到实际存在的relu/threshold值或者在不存在值时或者没有使用get.variable()来创建时将弹出异常。  
还有一种办法是，可以设置reuse在with代码块中：

In [30]:
with tf.variable_scope('relu') as scope:
    scope.reuse_variables()
    threshold = tf.get_variable('threshold')

现在你就可以如下共享变量了:

In [32]:
def relu(X):
    with tf.variable_scope('relu', reuse=True):
        threshold = tf.get_variable('threshold')
        w_shape = (int(X.get_shape()[1]), 1)
        w = tf.Variable(tf.random_normal(w_shape), name='weights')
        b = tf.Variable(0.0, name='bias')
        z = tf.add(tf.matmul(X, w), b, name='z')
        return tf.maximum(z, threshold, name='max')

tf.reset_default_graph()
sess = tf.InteractiveSession()

X = tf.placeholder(tf.float32, shape=(None, n_features), name='X')
with tf.variable_scope('relu'):  #create the variable
    threshold = tf.get_variable('threshold', shape=(),
                               initializer=tf.constant_initializer(0.0))
relus = [relu(X) for relu_index in range(5)]
output = tf.add_n(relus, name='output')

file_writer = tf.summary.FileWriter(logdir, sess.graph)

sess.close()
file_writer.close()

这里首先定义了relu函数，创建了relu/threshold变量。并且创建了5个relu，其中重用了变量。  
可惜的是这里阈值变量要在relu函数外创建。为了解决这点，下面的代码在relu函数第一次调用的时候就创建阈值变量，并在后面复用他。这里第一次使用时reuse是否的，而后面几次修改为真。

In [33]:
def relu(X):
    threshold = tf.get_variable('threshold',shape=(),
                               initializer=tf.constant_initializer(0.0))
    w_shape = (int(X.get_shape()[1]), 1)
    w = tf.Variable(tf.random_normal(w_shape), name='weights')
    b = tf.Variable(0.0, name='bias')
    z = tf.add(tf.matmul(X, w), b, name='z')
    return tf.maximum(z, threshold, name='max')

tf.reset_default_graph()
sess = tf.InteractiveSession()

X = tf.placeholder(tf.float32, shape=(None, n_features), name='X')
relus = []
for relu_index in range(5):
    with tf.variable_scope('relu', reuse=(relu_index >= 1)) as scope:
        relus.append(relu(X))
output = tf.add_n(relus, name='output')

file_writer = tf.summary.FileWriter(logdir, sess.graph)

sess.close()
file_writer.close()